## Imports

In [1]:
# Copy this to all notebooks!
from qcodes.logger import start_all_logging
start_all_logging()

# Import qcodes and other necessary packages
import qcodes as qc
import numpy as np
import time

# Import device drivers 
from qcodes.instrument_drivers.QuantumDesign.DynaCoolPPMS import DynaCool
from qcodes.instrument_drivers.Keysight.Infiniium import Infiniium


from qcodes.dataset.measurements import Measurement
from qcodes.dataset.plotting import plot_by_id
from qcodes.dataset.database import initialise_database
from qcodes.dataset.experiment_container import (Experiment,
                                                 load_last_experiment,
                                                 new_experiment)
from qcodes.dataset.experiment_container import load_experiment_by_name
from qcodes.instrument.base import Instrument
from qcodes import Station

import matplotlib.pyplot as plt
import matplotlib
from time import sleep
from qcodes.dataset.data_export import flatten_1D_data_for_plot

from qcodes.utils.dataset.doNd import do1d,do2d


Logging hadn't been started.
Activating auto-logging. Current session state plus future input saved.
Filename       : C:\Users\lyn-ppmsmsr-01usr\.qcodes\logs\command_history.log
Mode           : append
Output logging : True
Raw input log  : False
Timestamping   : True
State          : active
Qcodes Logfile : C:\Users\lyn-ppmsmsr-01usr\.qcodes\logs\200303-496-qcodes.log
Activating auto-logging. Current session state plus future input saved.
Filename       : C:\Users\lyn-ppmsmsr-01usr\.qcodes\logs\command_history.log
Mode           : append
Output logging : True
Raw input log  : False
Timestamping   : True
State          : active


2020-03-03 14:42:55,240 ¦ py.warnings ¦ WARNING ¦ warnings ¦ _showwarnmsg ¦ 110 ¦ C:\Users\lyn-ppmsmsr-01usr\miniconda3\envs\qcodes\lib\site-packages\qcodes\dataset\database.py:13: UserWarning: The module `qcodes.dataset.database` is deprecated.
Public features are available at the import of `qcodes`.
Private features are available in `qcodes.dataset.sqlite.*` modules.
  warnings.warn('The module `qcodes.dataset.database` is deprecated.\n'



Qcodes Logfile : C:\Users\lyn-ppmsmsr-01usr\.qcodes\logs\200303-496-qcodes.log


## Station
(Need to load 3 Keithleys and 6 Lock-In Amps)

In [3]:
# Create station, instantiate instruments
Instrument.close_all()
path_to_station_file = "please replace this sentence with the address of the " \
                       "file//station.yaml"
# Here we load the station file.
station = Station()
station.load_config_file(path_to_station_file)

# Connect to ppms
#Instrument.find_instrument('ppms_cryostat')
ppms = DynaCool.DynaCool(name = "ppms_cryostat", address="TCPIP0::10.10.117.37::5000::SOCKET")
station.add_component(ppms)

# SRS
lockin_1 = station.load_instrument('lockin_1')
lockin_2 = station.load_instrument('lockin_2')
lockin_3 = station.load_instrument('lockin_3')
lockin_4 = station.load_instrument('lockin_4')
lockin_5 = station.load_instrument('lockin_5')
lockin_6 = station.load_instrument('lockin_6')

# DMMs
dmm_a = station.load_instrument('Keithley_A')
dmm_b = station.load_instrument('Keithley_B')
dmm_c = station.load_instrument('Keithley_C')

dmm_a.smua.volt(0) # Set voltages to 0
dmm_a.smub.volt(0) # Set voltages to 0

dmm_b.smua.volt(0) # Set voltages to 0
dmm_b.smub.volt(0) # Set voltages to 0

dmm_c.smua.volt(0) # Set voltages to 0
dmm_c.smub.volt(0) # Set voltages to 0

Connected to:  QuantumDesign dynacool (serial:N/A, firmware:N/A) in 0.02s
Connected to: Keithley Instruments Inc. 2614B (serial:4376445, firmware:3.2.2) in 0.02s
Connected to: Stanford_Research_Systems SR830 (serial:s/n91493, firmware:ver1.07) in 0.10s
Connected to: Stanford_Research_Systems SR830 (serial:s/n91797, firmware:ver1.07) in 0.05s


'SR830M'

In [4]:
for inst in station.components.values():
    inst.print_readable_snapshot()

ppms_cryostat:
	parameter           value
--------------------------------------------------------------------------------
IDN                  :	{'vendor': ' QuantumDesign', 'model': 'dynacool', 'ser...
chamber_state        :	sealed 
chamber_temperature  :	1.8916 (K)
field_approach       :	linear 
field_measured       :	-9.8738e-06 (T)
field_ramp           :	None (T)
field_rate           :	0 (T/s)
field_target         :	-9.8738e-06 (T)
field_tolerance      :	0.0005 (T)
magnet_state         :	holding 
temperature          :	1.8916 (K)
temperature_rate     :	0 (K/s)
temperature_setpoint :	1.7 (K)
temperature_settling :	fast settle 
temperature_state    :	chasing 
timeout              :	5 (s)
KL:
	parameter      value
--------------------------------------------------------------------------------
IDN             :	{'vendor': 'Keithley Instruments Inc.', 'model': '2614B', '...
display_settext :	None 
timeout         :	5 (s)
KL_smua:
	parameter                  value
---------------------

## DB File, Location and Functions

In [4]:
#config.save_to_home  ### Initialize database, make new measurement
%matplotlib notebook

from qcodes.dataset.database import get_DB_location
from qcodes import config
import os.path
import os

mainpath = 'C:/Users/MicrosoftQ/Desktop/Results'
config.current_config.core.db_location = os.path.join(mainpath,'frequency_test.db')
config.current_config
newpath = os.path.join(mainpath,'frequency_test') 
if not os.path.exists(newpath):
    os.makedirs(newpath)
figurepath = newpath
initialise_database()


# DIFFERENTIAL CONDUCTANCE FOR A TWO PROBE MEASUREMENT

ac_divider     = 1e-4
gain_curr_ampl = 1e6

def desoverh():
    volt_ampl = sr830.amplitude
    amplitude_curr = sr830.R
    dcond = (amplitude_curr()/gain_curr_ampl)/(volt_ampl()*ac_divider)/go
    return dcond     
    
try:
    sr830.add_parameter("diff_conductance", label="dI/dV", unit="2e^2/h", get_cmd = desoverh)
except KeyError:
    print("parameter already exists. Deleting. Try again")
    del sr830.parameters['diff_conductance']

    
    
    
# we create a dummy variable to control the number of times that we will repeat the measurement

def number(n):
    return n
    
try:
    keith.smua.add_parameter("counts", label="N", unit="times",
                               get_cmd=number)
except KeyError:
    print("parameter already exists. Deleting. Try again")
    del keith.smua.parameters['counts']
    
    
    
#------------------------------------------------------------------------
def wait_for_field():
    time.sleep(0.1)
    Magnet_state = ppms.magnet_state()
    while Magnet_state is not 'holding':
        #print('waiting for field')
        time.sleep(0.1)
        Magnet_state = ppms.magnet_state()
        #print('field ready')
    return

def wait_for_field_ramp():
    Magnet_state = ppms.magnet_state()
    while Magnet_state is not 'ramping':
        time.sleep(0.1)
        Magnet_state = ppms.magnet_state()
    return

def field_ready():
    return ppms.magnet_state() == 'holding'

def wait_for_temp():
    Temp_state = ppms.temperature_state()
    while Temp_state is not 'stable':
        time.sleep(1)
        Temp_state = ppms.temperature_state()
    return


#--------------------------------------------------------------------------
#fig, ax = plt.subplots(1)
#fig2, ax2 = plt.subplots(1)

Upgrading database; v0 -> v1: : 0it [00:00, ?it/s]
Upgrading database; v1 -> v2: 100%|██████████████████████████████████████████████████████████████| 1/1 [00:00<?, ?it/s]
Upgrading database; v2 -> v3: : 0it [00:00, ?it/s]
Upgrading database; v3 -> v4: : 0it [00:00, ?it/s]
Upgrading database; v4 -> v5: 100%|██████████████████████████████████████████████████████████████| 1/1 [00:00<?, ?it/s]
Upgrading database; v5 -> v6: : 0it [00:00, ?it/s]
Upgrading database; v7 -> v8: 100%|█████████████████████████████████████████████████████| 1/1 [00:00<00:00, 205.19it/s]


## Frequency test
(For any lock-in amp)

In [30]:
#   Rough scan

new_experiment(name='lockin_start_up', sample_name='devX_rough_scan')


# Use whichever lock-in you are interested (lockin_X) or whatever name has
frequency = lockin_1.frequency            
amplitude = lockin_1.R

f_min  = 15
f_max  = 140
Nfreq  = 100
Ntimes = 50

# f_min = 10 Hz  => t_c = 1/15*5 => t_c = 300 ms
lockin_1.time_constant(0.3)
tdelay = 1

dc_divider     = 1e-2
ac_divider     = 1e-4
gain_curr_ampl = 1e6

dmm_a.smua.output('on')                   # Turn on the bias channel
dmm_a.smub.output('on')                   # Turn on the gate channel

dmm_a.smub.volt(0.1e-3/dc_divider)        # Set a small DC signal ie 0.1mV
dmm_a.smub.volt(-20)                      # Set the gate on a very high resistance area (below the pinch-off)

do2d(frequency, f_min,   f_max,  Nfreq, tdelay,
    counts,     1    , Ntimes , Ntimes, 0.01,
    amplitude)


Starting experimental run with id: 24. 


24

In [ ]:
#   Fine scan

new_experiment(name='lockin_start_up', sample_name='devX_fine_scan')


# Use whichever lock-in you are interested (lockin_X) or whatever name has
frequency = lockin_1.frequency            
amplitude = lockin_1.R

f_min  = 58
f_max  = 72
Nfreq  = 50
Ntimes = 100

# f_min = 60 Hz  => t_c = 1/60*5 => t_c = 100 ms
lockin_1.time_constant(0.1)
tdelay = 0.5

dc_divider     = 1e-2
ac_divider     = 1e-4
gain_curr_ampl = 1e6

dmm_a.smua.output('on')                   # Turn on the bias channel
dmm_a.smub.output('on')                   # Turn on the gate channel

dmm_a.smub.volt(0.1e-3/dc_divider)        # Set a small DC signal ie 0.1mV
dmm_a.smub.volt(-20)                      # Set the gate on a very high resistance area (below the pinch-off)

do2d(frequency, f_min,   f_max,  Nfreq, tdelay,
    counts,     1    , Ntimes , Ntimes, 0.01,
    amplitude)
